### Detecting Objects 

Until now we have see the data of classifying by using the features. 
We have features as numerical values and if they are not then we convert them to numerics 

Now we enter the world of Computer vision. Or working with images and making a computer see.


So let's jump in 

### HOG Histogram of Oriented Gradients

**Feature Descriptor**

Takes useful information from the image and represents it in numeric form. 


Typically, a feature descriptor converts an image of size width x height x 3 (channels ) to a feature vector / array of length n. In the case of the HOG feature descriptor, the input image is of size 64 x 128 x 3 and the output feature vector is of length 3780.


**What is a useful feature?**

good features extracted from an image should be able to tell the difference between buttons and other circular objects like coins and car tires.

In the HOG feature descriptor, the distribution ( histograms ) of directions of gradients ( oriented gradients ) are used as features. Gradients ( x and y derivatives ) of an image are useful because the magnitude of gradients is large around edges and corners ( regions of abrupt intensity changes ) and we know that edges and corners pack in a lot more information about object shape than flat regions.

## HOG Steps

### Preprocessing : 

We need a fixed size of image to work with. In the original paper the image size was 64x128 cropped image

![](../images/hog-preprocessing.jpg)

### Calculate Gradients 

**What is a gradient?**

Gradient is the change. How much the values are changing. In the image gradient is how much a value is changed in one cell from another cell. 

This can be achieved with a simple operation of multiplication and addition with help of kernels as 

![](../images/gradient-kernels.jpg)

Now we get gradient in x and y directions so let's get their magnitude and directions as 

![](../images/gradient_eqn.png)

Let's see some value of the gradients 

![](../images/gradients.png)


Left : X gradient 
          Middle  : Y gradient
          Right : Magnitude 


At every pixel, the gradient has a magnitude and a direction. For color images, the gradients of the three channels are evaluated ( as shown in the figure above ). The magnitude of gradient at a pixel is the maximum of the magnitude of gradients of the three channels, and the angle is the angle corresponding to the maximum gradient.

### Calculate the Histogram of Gradients

In this step, the image is divided into 8×8 cells and a histogram of gradients is calculated for each 8×8 cells.

![](../images/hog-cells.png)


why we have divided the image into 8×8 cells. One of the important reasons to use a feature descriptor to describe a patch of an image is that it provides a compact representation. An 8×8 image patch contains 8x8x3 = 192 pixel values. The gradient of this patch contains 2 values ( magnitude and direction ) per pixel which adds up to 8x8x2 = 128 numbers. By the end of this section we will see how these 128 numbers are represented using a 9-bin histogram which can be stored as an array of 9 numbers. Not only is the representation more compact, calculating a histogram over a patch makes this represenation more robust to noise. Individual graidents may have noise, but a histogram over 8×8 patch makes the representation much less sensitive to noise.


But why 8×8 patch ? Why not 32×32 ? It is a design choice informed by the scale of features we are looking for. HOG was used for pedestrian detection initially. 8×8 cells in a photo of a pedestrian scaled to 64×128 are big enough to capture interesting features ( e.g. the face, the top of the head etc. ).

The histogram is essentially a vector ( or an array ) of 9 bins ( numbers ) corresponding to angles 0, 20, 40, 60 … 160.

Let us look at one 8×8 patch in the image and see how the gradients look.

![](../images/hog-cell-gradients-2.png)


On the right, we see the raw numbers representing the gradients in the 8×8 cells with one minor difference — the angles are between 0 and 180 degrees instead of 0 to 360 degrees. These are called “unsigned” gradients because a gradient and it’s negative are represented by the same numbers. In other words, a gradient arrow and the one 180 degrees opposite to it are considered the same. But, why not use the 0 – 360 degrees ? Empirically it has been shown that unsigned gradients work better than signed gradients for pedestrian detection.

The next step is to create a histogram of gradients in these 8×8 cells. The histogram contains 9 bins corresponding to angles 0, 20, 40 … 160.

The following figure illustrates the process. We are looking at magnitude and direction of the gradient of the same 8×8 patch as in the previous figure. A bin is selected based on the direction, and the vote ( the value that goes into the bin ) is selected based on the magnitude. Let’s first focus on the pixel encircled in blue. It has an angle ( direction ) of 80 degrees and magnitude of 2. So it adds 2 to the 5th bin. The gradient at the pixel encircled using red has an angle of 10 degrees and magnitude of 4. Since 10 degrees is half way between 0 and 20, the vote by the pixel splits evenly into the two bins.

![](../images/hog-histogram-1.png)

The contributions of all the pixels in the 8×8 cells are added up to create the 9-bin histogram. For the patch above, it looks like this
![](../images/histogram-8x8-cell.png)

### Block Normalization

we created a histogram based on the gradient of the image. Gradients of an image are sensitive to overall lighting. If you make the image darker by dividing all pixel values by 2, the gradient magnitude will change by half, and therefore the histogram values will change by half. Ideally, we want our descriptor to be independent of lighting variations. In other words, we would like to “normalize” the histogram so they are not affected by lighting variations.

![](../images/hog-16x16-block-normalization.gif)


while calculating HOG you can simply normalize the 9×1 histogram the same way we normalized the 3×1 vector above. It is not a bad idea, but a better idea is to normalize over a bigger sized block of 16×16. A 16×16 block has 4 histograms which can be concatenated to form a 36 x 1 element vector and it can be normalized just the way a 3×1 vector is normalized. The window is then moved by 8 pixels ( see animation ) and a normalized 36×1 vector is calculated over this window and the process is repeated.

### HOG Calculation

the 36×1 vectors are concatenated into one giant vector. What is the size of this vector ? 

64x128x3 - original 
64x128x2 - gradient

pixels - 64x128x3 
cells - 8x16
histogram per cell = 8x16x9
blocks - 

#blocks = 7 in x and 15 in y
1 block - 

###### 

In [2]:
7*15*36

3780

### References

1. https://www.learnopencv.com/histogram-of-oriented-gradients/
